In [ ]:
import mordred as md
import numpy as np
import optuna
import pandas as pd
import torch
import torch.nn as nn
import torch_geometric.nn as gnn
from catboost import CatBoostRegressor, Pool
from rdkit import Chem
from rdkit.Chem import MolFromSmiles, MolToSmiles
from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch_geometric.loader import DataLoader as GDataLoader
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import thread_map

from yellow_cards_workflow import (BASE_DIR, MODEL_NAMES, OUTPUT_DIR,
                               PROCESSED_DIR, SEED)
from yellow_cards_workflow.datasets import (FCD_Dataset, FCFP_Dataset,
                                        GNNIMDataset, get_dict_gnn_dataset,
                                        prepare_gnn_dataset, CNN_Dataset)
from yellow_cards_workflow.preprocessing import (get_clean_dataset,
                                             load_processed_data)
from yellow_cards_workflow.utils import (encode_smiles, generate_fingerprints,
                                     generate_rdkit_descriptors)
from yellow_cards_workflow.models import CNN, GNN, FCD, FCFP

In [ ]:
MOD_RATE = 10

## Loading Data

In [3]:
properties_df = pd.read_csv(
    BASE_DIR / f"data/processed/qm9-shuffle-{MOD_RATE}.csv", sep=";", index_col=0
)
properties_df

,mol_id,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,u0,u298,h298,g298,cv,u0_atom,u298_atom,h298_atom,g298_atom
0,gdb_1,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.478930,-40.476062,-40.475117,-40.498597,6.469,-395.999595,-398.643290,-401.014647,-372.471772
1,gdb_2,293.60975,293.541110,191.393970,1.6256,9.46,-0.2570,0.0829,0.3399,26.1563,0.034358,-56.525887,-56.523026,-56.522082,-56.544961,6.316,-276.861363,-278.620271,-280.399259,-259.338802
2,gdb_3,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,0.3615,19.0002,0.021375,-76.404702,-76.401867,-76.400922,-76.422349,6.002,-213.087624,-213.974294,-215.159658,-201.407171
3,gdb_4,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,-385.501997,-387.237686,-389.016047,-365.800724
4,gdb_5,0.00000,44.593883,44.593883,2.8937,12.99,-0.3604,0.0191,0.3796,48.7476,0.016601,-93.411888,-93.409370,-93.408425,-93.431246,6.278,-301.820534,-302.906752,-304.091489,-288.720028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133878,gdb_133879,3.52243,2.185850,1.902520,1.4529,77.63,-0.2229,0.0977,0.3206,796.9713,0.151939,-364.733608,-364.728220,-364.727276,-364.762857,24.820,-1802.904893,-1815.522844,-1825.598756,-1678.637403
133879,gdb_133880,3.56068,2.195510,1.920670,1.3774,73.24,-0.2427,0.0904,0.3331,777.3913,0.139809,-384.609506,-384.604306,-384.603362,-384.638652,23.968,-1737.300710,-1749.148080,-1758.630996,-1619.706151
133882,gdb_133883,3.67118,2.143140,1.895010,1.2480,73.60,-0.2233,0.0720,0.2953,780.3553,0.140458,-380.753918,-380.748619,-380.747675,-380.783148,23.972,-1667.045429,-1678.830048,-1688.312964,-1549.143391
133883,gdb_133884,3.52845,2.151310,1.865820,1.9576,77.40,-0.2122,0.0881,0.3003,803.1904,0.152222,-364.720374,-364.714974,-364.714030,-364.749650,24.796,-1794.600439,-1807.210860,-1817.286772,-1670.349892


In [4]:
supplier = Chem.SDMolSupplier(BASE_DIR / "data/input/gdb9.sdf")


In [5]:
molecules = np.array([x for x in tqdm(supplier)])
print(np.count_nonzero(molecules))

  0%|          | 0/133885 [00:00<?, ?it/s]

[10:48:00] Explicit valence for atom # 1 C, 5, is greater than permitted
[10:48:00] ERROR: Could not sanitize molecule ending on line 9097
[10:48:00] ERROR: Explicit valence for atom # 1 C, 5, is greater than permitted
[10:48:00] Explicit valence for atom # 1 C, 5, is greater than permitted
[10:48:00] ERROR: Could not sanitize molecule ending on line 35785
[10:48:00] ERROR: Explicit valence for atom # 1 C, 5, is greater than permitted
[10:48:01] Explicit valence for atom # 4 C, 5, is greater than permitted
[10:48:01] ERROR: Could not sanitize molecule ending on line 62866
[10:48:01] ERROR: Explicit valence for atom # 4 C, 5, is greater than permitted
[10:48:01] Explicit valence for atom # 2 C, 5, is greater than permitted
[10:48:01] ERROR: Could not sanitize molecule ending on line 66832
[10:48:01] ERROR: Explicit valence for atom # 2 C, 5, is greater than permitted
[10:48:01] Explicit valence for atom # 2 C, 5, is greater than permitted
[10:48:01] ERROR: Could not sanitize molecule en

132737


[10:48:02] Explicit valence for atom # 2 C, 5, is greater than permitted
[10:48:02] ERROR: Could not sanitize molecule ending on line 664156
[10:48:02] ERROR: Explicit valence for atom # 2 C, 5, is greater than permitted
[10:48:02] Explicit valence for atom # 2 C, 5, is greater than permitted
[10:48:02] ERROR: Could not sanitize molecule ending on line 664543
[10:48:02] ERROR: Explicit valence for atom # 2 C, 5, is greater than permitted
[10:48:02] Explicit valence for atom # 2 C, 5, is greater than permitted
[10:48:02] ERROR: Could not sanitize molecule ending on line 664576
[10:48:02] ERROR: Explicit valence for atom # 2 C, 5, is greater than permitted
[10:48:02] Explicit valence for atom # 2 C, 5, is greater than permitted
[10:48:02] ERROR: Could not sanitize molecule ending on line 664869
[10:48:02] ERROR: Explicit valence for atom # 2 C, 5, is greater than permitted
[10:48:02] Explicit valence for atom # 2 C, 5, is greater than permitted
[10:48:02] ERROR: Could not sanitize molecu

In [6]:
clean_mask = np.load(
    BASE_DIR/"data/processed/qm9-mask.npy"
)
molecules = molecules[clean_mask]

In [7]:
noniso_smiles = np.array(
    list(
        map(
            lambda x: Chem.MolToSmiles(x, isomericSmiles=False, canonical=True),
            tqdm(molecules),
        )
    )
)
smiles = np.array(
    list(
        map(
            lambda x: Chem.MolToSmiles(x, isomericSmiles=True, canonical=True),
            tqdm(molecules),
        )
    )
)
len(np.unique(noniso_smiles)), len(np.unique(smiles))

  0%|          | 0/132398 [00:00<?, ?it/s]

  0%|          | 0/132398 [00:00<?, ?it/s]

(132398, 132398)

In [8]:
molecular_properties = properties_df["g298_atom"].to_numpy() / (-1000)
len(molecular_properties)

132398

In [9]:
morgan_fingerprints, rdkit_fingerprints = generate_fingerprints(molecules)

Generating Morgan fingerprints


  0%|          | 0/132398 [00:00<?, ?it/s]

Generating RDKit fingerprints


  0%|          | 0/132398 [00:02<?, ?it/s]

In [10]:
rdkit_descriptors = generate_rdkit_descriptors(molecules)

Loading / Generating RDKit descriptors


  0%|          | 0/132398 [00:00<?, ?it/s]

(132398, 217)
Scaling RDKit descriptors to zero mean and unit variance


In [11]:
smiles_dict, encoded_smiles = encode_smiles(smiles)

  0%|          | 0/132398 [00:00<?, ?it/s]

In [12]:
gnn_num_fingerprints, gnn_fingerprints, gnn_mol_bonds = prepare_gnn_dataset(molecules)

  0%|          | 0/132398 [00:00<?, ?it/s]

Atom Dict 0
Hybdn Dict 0
FPs Dict 1980


## 5Fold Split

In [13]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

device(type='cuda')

In [14]:
SEED = 42

In [15]:
def train_fn(model, optim, loss_fn, epochs, train_dl, eval_dl, name):

    writer = SummaryWriter(log_dir=BASE_DIR / "logs" / name, flush_secs=15)

    torch.cuda.empty_cache()
    b_train = 1e5
    b_eval = 1e5

    bar = tqdm(range(epochs), leave=False, position=1)
    for epoch in bar:
        epoch_train_loss = model.train_fn(optim, loss_fn, train_dl)
        b_train = min(b_train, epoch_train_loss)

        epoch_eval_loss = model.eval_fn(loss_fn, eval_dl)
        b_eval = min(b_eval, epoch_eval_loss)

        bar.set_postfix_str(f"{epoch_train_loss:.3f}({b_train:.3f}) | {epoch_eval_loss:.3f}({b_eval:.3f})")

        writer.add_scalar("loss/train", epoch_train_loss, epoch)
        writer.add_scalar("loss/eval", epoch_eval_loss, epoch)
        if epoch_eval_loss <= b_eval:
            torch.save(model.state_dict(),
                       BASE_DIR / f"models/{name}.pth")
    bar.close()
    return b_train, b_eval

In [16]:
kf = KFold(5, shuffle=True, random_state=SEED)
predicted_values = {
    "smiles": [],
    "values": [],
    "cnn": [],
    "gnn": [],
    "fcfp": [],
    "fcd": [],
    "cb": [],
}
pbar = tqdm(kf.split(np.arange(len(smiles))), position=0, total=5)
for k, (unique_train_idx, unique_eval_idx) in enumerate(pbar):
    train_gnn_fingerprints, eval_gnn_fingerprints = (
        [gnn_fingerprints[i] for i in unique_train_idx],
        [gnn_fingerprints[i] for i in unique_eval_idx],
    )
    train_gnn_mol_bonds, eval_gnn_mol_bonds = (
        [gnn_mol_bonds[i] for i in unique_train_idx],
        [gnn_mol_bonds[i] for i in unique_eval_idx],
    )
    train_encoded_smiles, eval_encoded_smiles = (
        encoded_smiles[unique_train_idx],
        encoded_smiles[unique_eval_idx],
    )
    train_rdkit_descriptors, eval_rdkit_descriptors = (
        rdkit_descriptors[unique_train_idx],
        rdkit_descriptors[unique_eval_idx],
    )
    train_morgan_fingerprints, eval_morgan_fingerprints = (
        morgan_fingerprints[unique_train_idx],
        morgan_fingerprints[unique_eval_idx],
    )
    train_rdkit_fingerprints, eval_rdkit_fingerprints = (
        rdkit_fingerprints[unique_train_idx],
        rdkit_fingerprints[unique_eval_idx],
    )
    train_molecular_properties, eval_molecular_properties = (
        molecular_properties[unique_train_idx],
        molecular_properties[unique_eval_idx],
    )

    predicted_values["smiles"].extend(smiles[unique_eval_idx])
    predicted_values["values"].extend(eval_molecular_properties)

    # pbar.write("Split Finished")
    model = CNN(
        len(smiles_dict) + 1,
        n_conv_layers=3,
        kernel_size=5,
        conv_channels=512,
        n_lin_layers=3,
    ).to(device)
    eval_ds = CNN_Dataset(eval_encoded_smiles, eval_molecular_properties)
    eval_dl = DataLoader(
        eval_ds, batch_size=512, shuffle=False, num_workers=4, persistent_workers=True
    )
    train_ds = CNN_Dataset(train_encoded_smiles, train_molecular_properties)
    train_dl = DataLoader(
        train_ds, batch_size=512, shuffle=True, num_workers=4, persistent_workers=True
    )
    # pbar.write("CNN Loaded")
    b_train, b_eval = train_fn(
        model=model,
        optim=torch.optim.Adam(model.parameters(), lr=1e-4),
        loss_fn=nn.L1Loss(reduction="sum"),
        epochs=200,
        train_dl=train_dl,
        eval_dl=eval_dl,
        name=f"5fold-shuffle-QM9-{MOD_RATE}-cnn-{SEED}-{k}",
    )
    pbar.write(f"5Fold\t CNN\t {k}\t {b_train:.4f}\t\t {b_eval:.4f}")
    model.load_state_dict(
        torch.load(
            BASE_DIR / f"models/5fold-shuffle-QM9-{MOD_RATE}-cnn-{SEED}-{k}.pth",
            map_location=device,
            weights_only=True,
        )
    )
    predicted_values["cnn"].extend(
        model.eval_fn(nn.L1Loss(reduction="sum"), eval_dl, return_predictions=True)
    )

    train_gnn_dataset = GNNIMDataset(
        get_dict_gnn_dataset(
            train_gnn_fingerprints, train_gnn_mol_bonds, train_molecular_properties
        )
    )
    eval_gnn_dataset = GNNIMDataset(
        get_dict_gnn_dataset(
            eval_gnn_fingerprints, eval_gnn_mol_bonds, eval_molecular_properties
        )
    )
    model = GNN(
        gnn_num_fingerprints,
        embed_fingerprints=16,
        n_conv_layers=9,
        conv_channels=1024,
        n_lin_layers=10,
    ).to(device)
    eval_dl = GDataLoader(
        eval_gnn_dataset,
        batch_size=128,
        shuffle=False,
        num_workers=4,
        persistent_workers=True,
    )
    train_dl = GDataLoader(
        train_gnn_dataset,
        batch_size=128,
        shuffle=True,
        num_workers=4,
        persistent_workers=True,
    )
    b_train, b_eval = train_fn(
        model=model,
        optim=torch.optim.Adam(model.parameters(), lr=1e-4),
        loss_fn=nn.L1Loss(reduction="sum"),
        epochs=200,
        train_dl=train_dl,
        eval_dl=eval_dl,
        name=f"5fold-shuffle-QM9-{MOD_RATE}-gnn-{SEED}-{k}",
    )

    pbar.write(f"5Fold\t GNN\t {k}\t {b_train:.4f}\t\t {b_eval:.4f}")
    model.load_state_dict(
        torch.load(
            BASE_DIR / f"models/5fold-shuffle-QM9-{MOD_RATE}-gnn-{SEED}-{k}.pth",
            map_location=device,
            weights_only=True,
        )
    )
    predicted_values["gnn"].extend(
        model.eval_fn(nn.L1Loss(reduction="sum"), eval_dl, return_predictions=True)
    )

    model = FCFP(n_layers=7, hidden_wts=4096).to(device)
    eval_ds = FCFP_Dataset(
        eval_morgan_fingerprints, eval_rdkit_fingerprints, eval_molecular_properties
    )
    eval_dl = DataLoader(
        eval_ds, batch_size=512, shuffle=False, num_workers=4, persistent_workers=True
    )
    train_ds = FCFP_Dataset(
        train_morgan_fingerprints, train_rdkit_fingerprints, train_molecular_properties
    )
    train_dl = DataLoader(
        train_ds, batch_size=512, shuffle=True, num_workers=4, persistent_workers=True
    )
    b_train, b_eval = train_fn(
        model=model,
        optim=torch.optim.Adam(model.parameters(), lr=1e-4),
        loss_fn=nn.L1Loss(reduction="sum"),
        epochs=250,
        train_dl=train_dl,
        eval_dl=eval_dl,
        name=f"5fold-shuffle-QM9-{MOD_RATE}-fcfp-{SEED}-{k}",
    )
    pbar.write(f"5Fold\t FCFP\t {k}\t {b_train:.4f}\t\t {b_eval:.4f}")
    model.load_state_dict(
        torch.load(
            BASE_DIR / f"models/5fold-shuffle-QM9-{MOD_RATE}-fcfp-{SEED}-{k}.pth",
            map_location=device,
            weights_only=True,
        )
    )
    predicted_values["fcfp"].extend(
        model.eval_fn(nn.L1Loss(reduction="sum"), eval_dl, return_predictions=True)
    )

    model = FCD(n_layers=7, hidden_wts=4096).to(device)
    eval_ds = FCD_Dataset(eval_rdkit_descriptors, eval_molecular_properties)
    eval_dl = DataLoader(
        eval_ds, batch_size=512, shuffle=False, num_workers=4, persistent_workers=True
    )
    train_ds = FCD_Dataset(train_rdkit_descriptors, train_molecular_properties)
    train_dl = DataLoader(
        train_ds, batch_size=512, shuffle=True, num_workers=4, persistent_workers=True
    )
    b_train, b_eval = train_fn(
        model=model,
        optim=torch.optim.Adam(model.parameters(), lr=1e-4),
        loss_fn=nn.L1Loss(reduction="sum"),
        epochs=250,
        train_dl=train_dl,
        eval_dl=eval_dl,
        name=f"5fold-shuffle-QM9-{MOD_RATE}-fcd-{SEED}-{k}",
    )
    pbar.write(f"5Fold\t FCD\t {k}\t {b_train:.4f}\t\t {b_eval:.4f}")
    model.load_state_dict(
        torch.load(
            BASE_DIR / f"models/5fold-shuffle-QM9-{MOD_RATE}-fcd-{SEED}-{k}.pth",
            map_location=device,
            weights_only=True,
        )
    )
    predicted_values["fcd"].extend(
        model.eval_fn(nn.L1Loss(reduction="sum"), eval_dl, return_predictions=True)
    )

    eval_ds = Pool(
        np.hstack(
            [
                eval_morgan_fingerprints,
                eval_rdkit_fingerprints,
                eval_rdkit_descriptors,
            ]
        ),
        eval_molecular_properties,
    )
    trn_ds = Pool(
        np.hstack(
            [
                train_morgan_fingerprints,
                train_rdkit_fingerprints,
                train_rdkit_descriptors,
            ]
        ),
        train_molecular_properties,
    )
    model = CatBoostRegressor(
        loss_function="MAE",
        task_type="GPU",
        devices="0",
        metric_period=20,
        learning_rate=0.00116,
        depth=8,
        iterations=10000,
        use_best_model=True,
        silent=True,
        allow_writing_files=False,
    )
    model.fit(trn_ds, eval_set=eval_ds, plot=False)
    model.save_model(BASE_DIR / f"models/5fold-shuffle-QM9-{MOD_RATE}-cb-{SEED}-{k}.cb")
    b_train = model.get_best_score().get("learn", {"MAE": np.nan})["MAE"]
    b_eval = model.get_best_score().get("validation", {"MAE": np.nan})["MAE"]
    pbar.write(f"5Fold\t CB\t {k}\t {b_train:.4f}\t\t {b_eval:.4f}")
    model = CatBoostRegressor().load_model(
        BASE_DIR / f"models/5fold-shuffle-QM9-{MOD_RATE}-cb-{SEED}-{k}.cb"
    )
    predicted_values["cb"].extend(np.stack(model.predict(eval_ds)))

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

5Fold	 CNN	 0	 0.0286		 0.0282


  0%|          | 0/105918 [00:00<?, ?it/s]

  0%|          | 0/26480 [00:00<?, ?it/s]

...

5Fold	 GNN	 4	 0.0112		 0.0350


  0%|          | 0/250 [00:00<?, ?it/s]

5Fold	 FCFP	 4	 0.0039		 0.0350


  0%|          | 0/250 [00:00<?, ?it/s]

5Fold	 FCD	 4	 0.0159		 0.0286
5Fold	 CB	 4	 0.0264		 0.0285


In [17]:
predicted_df = pd.DataFrame(predicted_values)
predicted_df.to_csv(
    BASE_DIR / f"data/processed/qm9-shuffle-predictions-{MOD_RATE}-{SEED}.csv",
    sep=";",
)